In [ ]:
!pip install snowflake-connector-python

In [ ]:
!pip install snowflake-connector-python pandas sqlalchemy snowflake-sqlalchemy

In [ ]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

engine = create_engine(URL(
    account='',
    user='',
    password='',
    database='',
    schema='',
    warehouse=''
))

In [ ]:
# Use with pandas
import pandas as pd
df = pd.read_sql("SELECT * FROM STOCK_FINAL_CLEANED", engine)  # Cleaner than raw connector
engine.dispose()  # Explicit cleanup
df.head()

,date,open,high,low,close,volume,dividends,stocksplits,ticker,money_flow_idx,...,ich_tenkan_sen,ich_kijun_sen,comm_channel_idx,stocchastic_osci,fib_23_6_retrace_lvl,fib_38_2_retrace_lvl,fib_61_8_retrace_lvl,std_dev,exp_mov_avg_12,exp_mov_avg_26
0,2024-01-26,328.550,323.757,330.109,323.550,375676,0.0,0.0,HUBB,62.173,...,325.330,322.227,68.702,47.131,397.538,353.279,281.737,4.868,320.675,316.408
1,2024-01-26,63.071,63.120,63.745,62.798,4595400,0.0,0.0,DD,24.232,...,68.185,69.562,-144.022,-5.928,79.159,73.264,63.736,4.213,68.434,70.196
2,2024-01-26,121.072,119.673,121.451,119.459,749000,0.0,0.0,AWK,46.420,...,123.599,125.158,-108.515,-10.903,161.747,152.399,137.290,3.638,122.180,124.050
3,2024-01-26,139.621,139.917,140.164,138.891,1910200,0.0,0.0,ROST,83.742,...,137.563,137.060,159.248,82.038,135.352,122.741,102.355,1.821,135.729,134.002
4,2024-01-26,130.012,136.064,136.809,130.012,5350300,0.0,0.0,COF,51.595,...,129.585,129.585,160.189,53.293,180.130,161.763,132.073,2.486,126.431,124.186


In [ ]:
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stocksplits', 'ticker', 'money_flow_idx', 'on_balance_volume',
       'accumu_dist_line', 'chaikin_money_flow', 'vol_wtd_avg_price',
       'simple_moving_avg', 'relative_strength_idx', 'price_vol_trend',
       'vol_rate_of_change', 'mov_avg_conver_divergence', 'bb_mid', 'bb_upper',
       'bb_lower', 'avg_true_range', 'rsi_volume', 'avg_direc_idx',
       'ich_tenkan_sen', 'ich_kijun_sen', 'comm_channel_idx',
       'stocchastic_osci', 'fib_23_6_retrace_lvl', 'fib_38_2_retrace_lvl',
       'fib_61_8_retrace_lvl', 'std_dev', 'exp_mov_avg_12', 'exp_mov_avg_26'],
      dtype='object')

In [ ]:
df.shape

(498989, 35)

In [ ]:
df.dtypes

date                          object
open                         float64
high                         float64
low                          float64
close                        float64
volume                         int64
dividends                    float64
stocksplits                  float64
ticker                        object
money_flow_idx               float64
on_balance_volume              int64
accumu_dist_line             float64
chaikin_money_flow           float64
vol_wtd_avg_price            float64
simple_moving_avg            float64
relative_strength_idx        float64
price_vol_trend              float64
vol_rate_of_change           float64
mov_avg_conver_divergence    float64
bb_mid                       float64
bb_upper                     float64
bb_lower                     float64
avg_true_range               float64
rsi_volume                   float64
avg_direc_idx                float64
ich_tenkan_sen               float64
ich_kijun_sen                float64
c

In [ ]:
!pip install langchain langchain-community langchain-openai pinecone-client sqlalchemy tiktoken python-dotenv

In [ ]:
pip uninstall langchain-core langchain-text-splitters langchain-openai -y

Found existing installation: langchain-core 0.1.28
Uninstalling langchain-core-0.1.28:
  Successfully uninstalled langchain-core-0.1.28
Found existing installation: langchain-text-splitters 0.0.1
Uninstalling langchain-text-splitters-0.0.1:
  Successfully uninstalled langchain-text-splitters-0.0.1
Found existing installation: langchain-openai 0.0.8
Uninstalling langchain-openai-0.0.8:
  Successfully uninstalled langchain-openai-0.0.8
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install langchain-core==0.3.56 langchain-text-splitters==0.3.7 langchain-openai==0.3.14

  Using cached langchain_core-0.3.56-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_openai-0.3.14-py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_core-0.3.56-py3-none-any.whl (437 kB)
Using cached langchain_openai-0.3.14-py3-none-any.whl (62 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip list | grep langchain

langchain                               0.3.21
langchain-aws                           0.2.10
langchain-community                     0.3.20
langchain-core                          0.3.56
langchain-openai                        0.3.14
langchain-pinecone                      0.2.5
langchain-tests                         0.3.19
langchain-text-splitters                0.3.7
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from getpass import getpass
import snowflake.connector
from snowflake.connector.pandas_tools import pd_writer
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import time

# ----------------------------
# 1. Configuration Setup
# ----------------------------
load_dotenv()

# Configure API keys
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = getpass("Enter Pinecone API key: ")
if "SNOWFLAKE_PASSWORD" not in os.environ:
    os.environ["SNOWFLAKE_PASSWORD"] = getpass("Enter Snowflake password: ")

# Snowflake credentials
SNOWFLAKE_ACCOUNT = "VL15725.us-east-2.aws"  # Your account locator
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER", "ManiKrishna9")
SNOWFLAKE_DATABASE = "STOCK_DATABASE"
SNOWFLAKE_SCHEMA = "PUBLIC"
SNOWFLAKE_WAREHOUSE = "COMPUTE_WH"


/opt/conda/lib/python3.12/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


Enter OpenAI API key:  ········
Enter Pinecone API key:  ········
Enter Snowflake password:  ········


In [ ]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from langchain.utilities import SQLDatabase

engine = create_engine(URL(
    account='',  # Corrected account format
    user='',
    password='',
    database='',
    schema='PUBLIC',
    warehouse='COMPUTE_WH'
))

# Step 2: Pass the engine to SQLDatabase
db = SQLDatabase(engine)


In [ ]:
!pip install langchain langchain-openai langchain-community snowflake-connector-python snowflake-sqlalchemy


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL


# 3. Create SQL Toolkit with LLM
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)  # <-- This was missing

# 4. Create Agent
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,  # <-- Pass toolkit instead of raw db
    agent_type="openai-tools",
    verbose=True
)

# 5. Run Query
response = agent.run("What was the daily closing average for MSFT in 2024?")
print(response)

/tmp/ipykernel_897/1643763510.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What was the daily closing average for MSFT in 2024?")




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
#!pip uninstall -y langchain-community pinecone-client
#!pip install langchain-pinecone pinecone-client

In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore  # Updated import
from langchain_openai import OpenAIEmbeddings
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

# 1. Initialize Pinecone (v3+)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# 2. Get the index
index_name = "agrifarmrag"
pinecone_index = pc.Index(index_name)  # Correct v3+ index object

# 3. Initialize Vector Store (NEW WAY)
vectorstore = PineconeVectorStore(
    pinecone_index,  # Directly pass the index
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    text_key="text"  # Your metadata field containing text
)


In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor

# Pinecone retriever tool
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever,
    name="financial_context",
    description="Provides technical indicator context for SQL queries"
)

# SQL Toolkit
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Combine tools
tools = toolkit.get_tools() + [retriever_tool]

# Create agent
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    extra_tools=tools,
    agent_type="openai-tools",
    verbose=True
)

In [ ]:
def get_enhanced_response(question: str) -> str:
    # Step 1: Retrieve relevant technical context
    context = retriever.get_relevant_documents(question)

    # Step 2: Generate SQL with context
    response = agent.run(
        f"""Use this context: {context}
        Answer: {question}
        Prefer recent data (last 6 months) unless specified.
        """
    )

    return response

In [ ]:
# Example query
response = agent.run("Show me stocks with RSI > 70 and volume spike last week")
print(response)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("tell me which sector do i invest")
print(response)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/
Invoking: `sql_db_query` with `{'query': 'SELECT gics_sector, COUNT(*) AS company_count FROM companies_sp500 GROUP BY gics_sector ORDER BY company_count DESC LIMIT 1'}`


[(

In [ ]:
# Example query
response = agent.run("i wanted to invest my money where to invest?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("yes")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("which company is best ti invest in stocks?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("can you tell, based on prevoius day perfomance, which compnay is liekly to fall in stiock market?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("can you give me graph for markets that might fall in stcok values?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'nyse_stock_data, stock_final'}`



CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(16777216)
)

/*
3 rows from nyse_stock_data table:
date	open	high	low	close	volume	dividends	stocksplits	ticker
2021-04-23 00:00:00	131.0233675826983	133.07774353027344	133.5061495530119	130.9552053260634	1495200	0.0	0.0	A
2021-04-26 00:00:00	133.59378847133183	133.72035217285156	134.19744339798402	133.0777431406254	1287400	0.0	0.0	A
2021-04-27 00:00:00	133.32117858118303	132.88304138183594	133.8177469495319	132.63963017590524	1752900	0.0	0.0	A
*/


CREATE TABLE stock_final (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("i have no idea whetre to invest my money, can you helkp me, which comapny and sector to invest to get profits?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500,nyse_stock_data'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	s

In [ ]:
# Example query
response = agent.run("based on few technical indicators analyze apple and microsoft stocks, and suggest if i should buy or sell")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("which stock has high volatility?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("I have 5000 dollars, please invsest in the stiock that has highest dividend and what will be my overall portfolio after 5 years? ")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("I have 5000 dollars, please invsest in the stiock that has highest dividend and what will be my overall portfolio after 5 years? ")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_final_cleaned'}`



CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(50), 
	money_flow_idx FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	stocchastic_osci FLOAT, 
	fib_23_6_retrace_lvl FLOAT, 
	fib_38_2_retrace

In [ ]:
# Example query
response = agent.run("how old are you?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleanedI don't know.

> Finished chain.
I don't know.


In [ ]:
# Example query
response = agent.run("how is everone at home?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("who am i?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("remember i am divya vemula, passinate about data science and machine leraning, aiming to become next ceo to my nstart up")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_data_cleaned_divya'}`



CREATE TABLE stock_data_cleaned_divya (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(16777216), 
	"Money_Flow" FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	"Stochastic_Osc" FLOAT, 
	"Fib_23_6" FLOAT, 
	"Fi

In [ ]:
# Example query
response = agent.run("now, tell me who am i?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE nyse_stock_data (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 


In [ ]:
# Example query
response = agent.run("i tol;d you remeber that i am Divya Vemula! It's great to hear about your passion for data science and machine learning, and your aspirations to become the CEO of your startup., whenever iu ask you that who am from now on you have to say aboiut me")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'stock_data_cleaned_divya'}`



CREATE TABLE stock_data_cleaned_divya (
	date TIMESTAMP_NTZ, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	stocksplits FLOAT, 
	ticker VARCHAR(16777216), 
	"Money_Flow" FLOAT, 
	on_balance_volume DECIMAL(38, 0), 
	accumu_dist_line FLOAT, 
	chaikin_money_flow FLOAT, 
	vol_wtd_avg_price FLOAT, 
	simple_moving_avg FLOAT, 
	relative_strength_idx FLOAT, 
	price_vol_trend FLOAT, 
	vol_rate_of_change FLOAT, 
	mov_avg_conver_divergence FLOAT, 
	bb_mid FLOAT, 
	bb_upper FLOAT, 
	bb_lower FLOAT, 
	avg_true_range FLOAT, 
	rsi_volume FLOAT, 
	avg_direc_idx FLOAT, 
	ich_tenkan_sen FLOAT, 
	ich_kijun_sen FLOAT, 
	comm_channel_idx FLOAT, 
	"Stochastic_Osc" FLOAT, 
	"Fib_23_6" FLOAT, 
	"Fi

In [ ]:
# Example query
response = agent.run("tech stock vs non tech stocks?")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


companies_sp500, nyse_stock_data, stock_data_cleaned_divya, stock_final, stock_final_cleaned
Invoking: `sql_db_schema` with `{'table_names': 'companies_sp500,stock_final_cleaned'}`



CREATE TABLE companies_sp500 (
	ticker VARCHAR(10), 
	company_name VARCHAR(100), 
	gics_sector VARCHAR(50), 
	sub_industry VARCHAR(100), 
	headquarters VARCHAR(100), 
	founded VARCHAR(200)
)

/*
3 rows from companies_sp500 table:
ticker	company_name	gics_sector	sub_industry	headquarters	founded
MMM	3M (Minnesota Mining and Manufacturing Company)	Industrials	Industrial Conglomerates	Saint Paul, Minnesota	1902
AOS	A. O. Smith	Industrials	Building Products	Milwaukee, Wisconsin	1916
ABT	Abbott Laboratories	Health Care	Health Care Equipment	North Chicago, Illinois	1888
*/


CREATE TABLE stock_final_cleaned (
	date DATE, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume DECIMAL(38, 0), 
	dividends FLOAT, 
	st